In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader,Dataset
import torchvision
from torchvision.transforms import ToTensor
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets
import numpy as np

In [2]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [3]:
# class RNN (nn.Module):
#     def __init__(self,input_size:int):
#         super(RNN,self).__init__()
#         self.rnn=nn.RNN(input_size,hidden_size=256,num_layers=1,batch_first=True)    


#     def forward(self,x):
#         a0=torch.zeros(1,x.size(0),256).to(device)
#         out,n=self.rnn(x,a0)
#         # out=out.reshape(out.shape[0],-1)
#         return out,n
        
        

In [4]:
# model=RNN(28).to(device)
# x=torch.randn(64,28,28).to(device)
# x.size(0)
# model(x)[0].shape

In [5]:
# model(x)[1].shape

In [6]:
class LSTM (nn.Module):
    def __init__(self,input_size:int,num_classes:int,depth:int):
        super(LSTM,self).__init__()
        self.lstm=nn.LSTM(input_size,hidden_size=256,num_layers=depth,batch_first=True)
        self.fc=nn.Linear(in_features=256*28,out_features=num_classes)


    def forward(self,x):
        a0=torch.zeros(depth,x.size(0),256).to(device)
        c0=torch.zeros(depth,x.size(0),256).to(device)
        out,_=self.lstm(x,(a0,c0))
        out=out.reshape(out.shape[0],-1)
        out=self.fc(out)
        return out
        
        

In [7]:
# model=RNN(28,10,2).to(device)
# x=torch.randn(64,28,28).to(device)
# model(x)[0].shape

In [8]:
input_size=28
num_classes=10
depth=2
batch_size=64
epochs=2
lr=0.0001


In [9]:
train_dataset=datasets.MNIST(root='data',train=True,transform=transforms.ToTensor(),download=True)

test_dataset=datasets.MNIST(root='data',train=False,transform=transforms.ToTensor(),download=True)

train_data_loader=DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)

test_data_loader=DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)



In [10]:
model=LSTM(input_size,num_classes,depth).to(device)

In [11]:
from torchinfo import summary

summary(model,input_size=[1,28,28])

Layer (type:depth-idx)                   Output Shape              Param #
LSTM                                     [1, 10]                   --
├─LSTM: 1-1                              [1, 28, 256]              819,200
├─Linear: 1-2                            [1, 10]                   71,690
Total params: 890,890
Trainable params: 890,890
Non-trainable params: 0
Total mult-adds (M): 23.01
Input size (MB): 0.00
Forward/backward pass size (MB): 0.06
Params size (MB): 3.56
Estimated Total Size (MB): 3.62

In [12]:
loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(params=model.parameters(),lr=lr)

In [13]:
def acc_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc
     

In [14]:
from tqdm.auto import tqdm
train_loss=0
train_acc=0

for epoch in tqdm(range(epochs)):
  for batch_idx,(x,y) in enumerate(train_data_loader):
      x=x.to(device).squeeze(1)
      y=y.to(device)
      model.train()
      scores=model(x).to(device)
      loss=loss_fn(scores,y)
      train_loss+=loss
      train_acc+=acc_fn(y,scores.argmax(dim=1))

      model.zero_grad()
      loss.backward()
      optimizer.step()

  train_loss/=len(train_data_loader)
  train_acc/=len(train_data_loader)

      
  test_loss,test_acc,baseline_acc=0,0,0
  model.eval()
  with torch.inference_mode():
    for x_test,y_test in test_data_loader:
      x_test=x_test.to(device).squeeze(1)
      y_test=y_test.to(device)
      test_pred=model(x_test)

      test_loss+=loss_fn(test_pred,y_test)
      test_acc+=acc_fn(y_true=y_test,y_pred=test_pred.argmax(dim=1))
      baseline_acc+=acc_fn(y_test,torch.randint_like(y_test,high=9))

    test_loss/=len(test_data_loader)
    test_acc/=len(test_data_loader)
    baseline_acc/=len(test_data_loader)
  print(f"\n train loss:{train_loss:.4f} | train acc:{train_acc:.4f} | test loss:{test_loss:.4f} | test acc:{test_acc:.4f} |baseline_acc:{baseline_acc:.4f} ")
   

      

  0%|          | 0/2 [00:00<?, ?it/s]


 train loss:0.5635 | train acc:82.1662 | test loss:0.2076 | test acc:93.4912 |baseline_acc:9.9025 

 train loss:0.1749 | train acc:94.6722 | test loss:0.1391 | test acc:95.5514 |baseline_acc:10.2209 
